<a href="https://colab.research.google.com/github/eisbetterthanpi/vision/blob/main/resnet_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title download
# original 10k
# # https://drive.google.com/file/d/1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q/view?usp=sharing
# !gdown 1NkCNecLpFG3i7bo3Vl9RQSwzBpSRQ29q -O gsv.zip
# !unzip /content/gsv.zip -d /content
# import shutil
# shutil.rmtree('/content/google_street_view/meta_data', ignore_errors=True) # delete the meta_data folder

# # clip cleaned
# # https://drive.google.com/file/d/1-xcHyVAMeTkY7SUdUyQRVAn_FFXPSDGB/view?usp=share_link
# !gdown 1-xcHyVAMeTkY7SUdUyQRVAn_FFXPSDGB -O gsv.zip
# !unzip /content/gsv.zip -d /
# !rm -R /content/gsv/.ipynb_checkpoints
# !rm -R /content/gsv/01/.ipynb_checkpoints
# !rm -R /content/gsv/02/.ipynb_checkpoints
# !rm -R /content/gsv/03/.ipynb_checkpoints
# !rm -R /content/gsv/04/.ipynb_checkpoints
# !rm -R /content/gsv/05/.ipynb_checkpoints
# !rm -R /content/gsv/06/.ipynb_checkpoints

# # gsv 70k
# # https://drive.google.com/file/d/1-7ZC29k4VxXQkpnOuLfj7Ag_SFTM4LV8/view?usp=share_link
# !gdown 1-7ZC29k4VxXQkpnOuLfj7Ag_SFTM4LV8 -O gsv.zip
# !unzip /content/gsv.zip -d /

# # # !ls
# !ls -a /content/gsv70k
# !rm -R /content/gsv70k/.ipynb_checkpoints
# # # !rm -R /content/gsv/06/.ipynb_checkpoints

# # 70k+gmap
# # https://drive.google.com/file/d/1-CZp7TbhJLeRQpbKQCyT8ofGg89Yt137/view?usp=sharing
!gdown 1-CZp7TbhJLeRQpbKQCyT8ofGg89Yt137 -O gsv.zip
!unzip /content/gsv.zip -d /
!rm -R /content/gsv70kg/.ipynb_checkpoints
!rm -R /content/gsv70kg/01/.ipynb_checkpoints
!rm -R /content/gsv70kg/02/.ipynb_checkpoints
!rm -R /content/gsv70kg/03/.ipynb_checkpoints
!rm -R /content/gsv70kg/04/.ipynb_checkpoints
!rm -R /content/gsv70kg/05/.ipynb_checkpoints
!rm -R /content/gsv70kg/06/.ipynb_checkpoints

# # https://bestasoff.medium.com/how-to-fine-tune-very-large-model-if-it-doesnt-fit-on-your-gpu-3561e50859af
!pip install bitsandbytes


In [ ]:
# @title torch augment
# https://github.com/facebookresearch/vicreg/blob/main/augmentations.py
import torch
import torchvision.transforms as transforms

class TrainTransform(object):
    def __init__(self):
        self.transform = transforms.RandomApply([transforms.Compose([
        # self.transform = transforms.Compose([
                transforms.RandomPerspective(distortion_scale=0.3, p=0.5), # me
                transforms.RandomResizedCrop((400,640), scale=(0.7, 1.0), ratio=(0.8, 1.25), interpolation=transforms.InterpolationMode.BICUBIC),
                transforms.RandomHorizontalFlip(p=0.5), # 0.5
                transforms.RandomApply([transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8,), # brightness=0.4, contrast=0.4, saturation=0.2, hue=0.1)], p=0.8
                transforms.RandomGrayscale(p=0.2), # 0.2
                # transforms.RandomChoice(transforms.ColorJitter , transforms.RandomGrayscale(p=1.)
                transforms.RandomApply([transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),], p=1.0),
                # transforms.RandomSolarize(threshold=130, p=0.5)
                transforms.RandomErasing(p=0.5, scale=(0.1, 0.11), ratio=(1,1), value=0, inplace=True), # default p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False
                # transforms.ToTensor(), # ToTensored at dataset level, no need to ToTensor again
                # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # normalised at dataset level. default 0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225
                ])
            ], p=0.5)

    def __call__(self, sample):
        dims = len(sample.shape)
        if dims==3: x1 = self.transform(sample) # same transforms per minibatch
        elif dims==4: x1 = transforms.Lambda(lambda x: torch.stack([self.transform(x_) for x_ in x]))(sample) # diff transforms per img in minibatch
        return x1


In [ ]:
# @title data weighted
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np

# dataset has PILImage images of range [0, 1], transform them to Tensors of normalized range [-1, 1]
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
# transform = transforms.Compose(transforms.ToTensor())

# dir='/content/gsv'
dir='/content/gsv70kg'

# data = datasets.ImageFolder(dir, transform=transform)
data = datasets.ImageFolder(dir, transform=None)
torch.manual_seed(0)

# split data manually so that can work with weighted random sampler
# train_data, test_data = torch.utils.data.random_split(data, [.85,.15])
# https://www.scaler.com/topics/pytorch/how-to-split-a-torch-dataset/
data_size = len(data)
indices = np.arange(data_size)
np.random.shuffle(indices)
split_index = int(np.floor(0.9 * data_size))
train_idx, test_idx = indices[:split_index], indices[split_index:]
train_data = torch.utils.data.Subset(data, train_idx)
test_data = torch.utils.data.Subset(data, test_idx)
targets = np.array(data.targets)
train_targets = targets[train_idx]
test_targets = targets[test_idx]


from torch.utils.data import Dataset
class DatasetWrap(Dataset):
    def __init__(self, dataset, transform=None):
        super(DatasetWrap, self).__init__()
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)
        
    def __getitem__(self, index):
        x, y = self.dataset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

# dataset wrapper in order to apply transforms to train data only
# train_data = DatasetWrap(train_data, TrainTransform()) # apply data augmentation to train dataset only
train_data = DatasetWrap(train_data, transform) # apply transform during training to use gpu
test_data = DatasetWrap(test_data, transform)

# use batch size 16 for resnet 152/ vit with grad accumulation
# can use batch size 64 for inception v3 without grad accumulation?
batch_size = 16 # 64/16
num_batches=int(np.ceil(len(train_data)/batch_size))

# oversampling
# https://stackoverflow.com/questions/62319228/number-of-instances-per-class-in-pytorch-dataset
from collections import Counter
class_count = torch.tensor(list(Counter(data.targets).values()))
weights=1./class_count
# weights=sum(class_count)/class_count
# print(weights)
# https://pytorch.org/docs/stable/data.html#torch.utils.data.WeightedRandomSampler

train_weight = weights[train_targets]
test_weight = weights[test_targets]
# train_sampler = torch.utils.data.WeightedRandomSampler(train_weight, len(train_weight))
# test_sampler = torch.utils.data.WeightedRandomSampler(test_weight, len(test_weight))
train_sampler = torch.utils.data.WeightedRandomSampler(train_weight, 70000)
test_sampler = torch.utils.data.WeightedRandomSampler(test_weight, 7000)
train_loader = torch.utils.data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size, num_workers=4, pin_memory=True)
data, train_data, test_data = None, None, None

# test oversampling: occurence of each class should be roughly equal
# c=0
# print(len(test_loader))
# # for batch, (x, y) in enumerate(train_loader):
# for batch, (x, y) in enumerate(test_loader):
#     print(torch.bincount(y)) # torch count number of elements with value in tensor
#     c+=1
#     if c>5: break

# display img from torch tensor
def imshow(img):
    img = img / 2 + 0.5  # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# dataiter = iter(train_loader)
# images, labels = next(dataiter)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# dataiter=None
# print(labels)

# dataiter = iter(test_loader)
# images, labels = next(dataiter)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# dataiter=None


In [ ]:
# @title resnet
import torch
import torch.nn as nn
from torchvision import models
# # https://pytorch.org/vision/0.12/models.html#id10
model = models.resnet152(weights='DEFAULT') # 18 34 50 101 152
num_ftrs = model.fc.in_features
model.fc = nn.Sequential( # og (fc): Linear(in_features=2048, out_features=1000, bias=True)
    nn.Linear(num_ftrs, 6),
    nn.Softmax(dim=1),
    )

# # model.mods = [module for k, module in model._modules.items()]
# # modules = [module for k, module in model._modules.items()]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
model = torch.compile(model.to(device))
# model = torch.compile(model.to(device),mode='reduce-overhead')
# model = torch.compile(model.to(device),mode='max-autotune')

# resnet152 batch16 compile gradacc nogradckpt lr3e-6,3e-5


In [ ]:
# @title vit
# https://arxiv.org/pdf/2010.11929.pdf
import torch
import torch.nn as nn
from torchvision import models
# https://pytorch.org/vision/main/models/vision_transformer.html
# https://pytorch.org/vision/main/models/generated/torchvision.models.vit_b_16.html#torchvision.models.vit_b_16
# model = models.vit_l_16(weights='DEFAULT') # small vit_b_16 vit_b_32 vit_l_16 vit_l_32 vit_h_14 big
# # VisionTransformer(image_size, patch_size, num_layers, num_heads, hidden_dim, mlp_dim)
# num_ftrs = model.heads.head.in_features
# # num_ftrs = model.heads[-1].in_features
# model.heads = nn.Sequential(
#     # nn.Dropout(0.2),
#     nn.Linear(num_ftrs, 6, bias=False),
#     nn.Softmax(dim=1),
#     )


!pip install timm
# https://github.com/huggingface/pytorch-image-models/issues/908
import timm
# model = timm.create_model('vit_base_patch16_224', pretrained=True)
model = timm.create_model('vit_base_patch16_224', img_size=(400, 640), pretrained=True)
# [print(x) for x in timm.list_models('vit*',pretrained=True)]
# https://huggingface.co/google/vit-base-patch16-224
# https://github.com/huggingface/pytorch-image-models/blob/main/timm/models/vision_transformer.py
# vit_base_patch16_224 compile,no ckpt # patch_size=16, embed_dim=768, depth=12, num_heads=12
# vit_base_patch16_384
# vit_large_patch16_224 explodesgpu # patch_size=16, embed_dim=1024, depth=24, num_heads=16
# vit_large_patch16_384 # patch_size=16, embed_dim=1024, depth=24, num_heads=16

# or fine tune huge
# vit_large_patch14_224 # patch_size=16, embed_dim=1024, depth=24, num_heads=16
# vit_large_patch16_384


num_ftrs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(num_ftrs, 6),
    nn.Softmax(dim=1),
    )
# model.set_grad_checkpointing()

# print(model.patch_embed.grid_size) # (25, 40)
# print(model.pos_embed.shape) # [1, 1001, 768]
# https://github.com/rwightman/pytorch-image-models/blob/master/timm/models/vision_transformer.py

# print(model)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
model = torch.compile(model.to(device))
# model = torch.compile(model.to(device),mode='reduce-overhead')
# model = torch.compile(model.to(device),mode='max-autotune')

# vit_base_patch16_224 batch16 maxcompile nockpt gradacc lr1e-5,1e-4


In [ ]:
# @title inception
import torch
import torch.nn as nn
from torchvision import models
# # https://pytorch.org/vision/0.12/models.html#id10
model = models.inception_v3(pretrained=True)
# https://discuss.pytorch.org/t/inception-v3-is-not-working-very-well/38296/16
# https://colab.research.google.com/github/CaoCharles/Deep-Learning-with-PyTorch/blob/master/2_Inception.ipynb
model.aux_logits = False
num_ftrs = model.fc.in_features # 2048
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 6), # og: (fc): Linear(in_features=2048, out_features=1000, bias=True)
    # nn.Linear(num_ftrs, 128), nn.ReLU(), nn.Linear(128, 6),
    nn.Softmax(dim=1),
    )
# pytorch "inception" v3 "gradient checkpointing" https://github.com/jianweif/OptimalGradCheckpointing

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = model.to(device)
model = torch.compile(model.to(device))
# model = torch.compile(model.to(device),mode='max-autotune')

# inception batch64 compile nogradacc nogradckpt lr3e-6,3e-5


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:00<00:00, 208MB/s]


In [ ]:
# @title try

# # check model's input and output dimensions are correct
# X = torch.rand(64, 3, 32, 32, device=device)
X = torch.rand(16, 3, 400, 640, device=device)
# X = torch.rand(16, 3, 224, 224, device=device)
model.eval()

# 224x224
# 16x16 / 32x32 patch
# -> 14x14=196 7x7=49 seq length
# 400x640 -> 25x40=1000 seq length


logits = model(X)

# modules = [module for k, module in model._modules.items()]
# for i,x in enumerate(modules):
#     print(i,x)

# logits = checkpoint_sequential(functions=modules, segments=1, input=X)

print(logits.shape)
# print(logits[0])
# print(logits[0].argmax(1))

pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(y_pred)



In [ ]:
# @title train test function
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()
# https://github.com/prigoyal/pytorch_memonger/blob/master/models/optimized/resnet_new.py
from torch.utils.checkpoint import checkpoint, checkpoint_sequential

trs=TrainTransform()
# train function with automatic mixed precision
def strain(dataloader, model, loss_fn, optimizer, scheduler=None, verbose=True):
    size = len(dataloader.dataset)
    model.train()
    loss_list = []
    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        with torch.cuda.amp.autocast(): # automatic mixed percision
            x = trs(x) # image augmentation during train time to use gpu
            pred = model(x) # default

            # modules = [module for k, module in model._modules.items()]
            # pred = checkpoint_sequential(functions=modules, segments=1, input=x) # gradient checkpointing for resnet and inception only
            # # # pred = checkpoint_sequential(functions=model.mods, segments=1, input=x)

            loss = loss_fn(pred, y) # /4 to scale by gradient accumulation_steps? no?

        scaler.scale(loss).backward()

        if ((batch + 1) % 4 == 0) or (batch + 1 == len(dataloader)): # gradient accumulation
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()

        loss_list.append(loss.item())
        # loss.backward()
        # optimizer.step()
        # if (batch) % (size//(10* len(x))) == 0:
        # if batch % ((dataloader.sampler.num_samples/dataloader.batch_size)//10) == 0:
        # loss, current = loss.item(), batch * len(x)
        loss, current = loss.item()/len(y), batch * len(x)
        # if verbose: print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        if verbose: print(f"loss: {loss:>7f} [{current:>5d}/{dataloader.sampler.num_samples:>5d}]")
    return loss_list

def test(dataloader, model, loss_fn, verbose=True):
    # size = len(dataloader.dataset)
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            x, y = X.to(device), y.to(device)
            pred = model(x)
            loss = loss_fn(pred, y)
            # predicted, actual = classes[pred[0].argmax(0)], classes[y]
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # test_loss /= num_batches
    # correct /= size
    test_loss /= dataloader.sampler.num_samples
    correct /= dataloader.sampler.num_samples
    # if verbose: print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    if verbose: print(f"Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f}")
    return correct, test_loss


In [ ]:
# @title LR range test
# gives insight into good LR range to use. 
# for accurate results, be sure to use a new model for range test; 
# also reset the model before training bec range test destroys the model
# 1cycle super convergencehttps://arxiv.org/pdf/1708.07120.pdf
# # cyclic lr https://arxiv.org/pdf/1506.01186.pdf
# Note the learning rate value when the accuracy starts to
# increase and when the accuracy slows, becomes ragged, or starts to fall

# one training run of the network for a few epochs
# pth='/content/lr.pth'
# torch.save(model.state_dict(), pth) # save temporary model for lr finding
# model.load_state_dict(torch.load("lr.pth"))
torch.autograd.set_detect_anomaly(True)
epochs=1
min_lr= 1e-6
max_lr= 1e-3 # 1e-2
# 152: 1e-7 - 1e-4      result 3e-7 - 3e-6
# inception: 1e-7 - 1e1      result 3e-7 - 3e-6

loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.AdamW(model.parameters(), lr=start_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01)
# optimizer = torch.optim.SGD(model.parameters(), lr=start_lr, momentum=0.9)
import bitsandbytes as bnb
optimizer = bnb.optim.AdamW(model.parameters(), lr=min_lr, betas=(0.9, 0.999), optim_bits=8)

num_batches=len(test_loader)
# num_batches=len(train_loader)

total_steps=int(num_batches*epochs)
# total_steps=int(np.ceil(num_batches/4)*epochs)

# min_lr* gamma^total_steps = max_lr
gamma = np.exp(np.log(max_lr/min_lr)/total_steps) # for scheduler step every optimizer step
lr_list=np.ones(total_steps)*min_lr*gamma**np.arange(total_steps)
# gamma = np.exp(np.log(max_lr/min_lr)/(total_steps*4)) # total_steps*4 bec grad accumulation, loss step 4x per lr step
# lr_list=np.ones(total_steps*4)*min_lr*gamma**np.arange(total_steps*4)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # 0.75(20)-0.9(100)
train_lst, test_lst=[],[]


for i in range(epochs):
    train_ls = strain(test_loader, model, loss_fn, optimizer, scheduler)    
    # train_ls = strain(train_loader, model, loss_fn, optimizer, scheduler)    
    train_lst.extend(train_ls)

# https://stackoverflow.com/a/53472966/13359815
from scipy.ndimage.filters import gaussian_filter1d
train_lstsm = gaussian_filter1d(train_lst, sigma=30)

import matplotlib.pyplot as plt
plt.plot(lr_list, train_lst)
plt.plot(lr_list, train_lstsm)
plt.xscale('log')
# plt.plot(train_lst)
# plt.plot(np.linspace(0,len(train_lst),len(test_lst)), test_lst)
plt.show()
plt.close()


In [ ]:
# @title wwwwwwwww
acc_lst, train_lst, test_lst=[],[],[]
loss_fn = nn.CrossEntropyLoss()

base_lr, max_lr = 3e-7, 3e-6 # resnet
# base_lr, max_lr = 1e-5, 1e-2 # vit
# base_lr, max_lr = 3e-6, 3e-5 # inception
# end_lr, start_lr = 1e-5, 1e-3 # 0.0001,0.1
tp=0
epochs = 5 #5 20
# optimizer = torch.optim.SGD(model.parameters(), lr = start_lr, momentum=0.9)
# optimizer = torch.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=3e-6)
import bitsandbytes as bnb # 8bit optimizer
optimizer = bnb.optim.AdamW(model.parameters(), lr=base_lr, betas=(0.9, 0.999), optim_bits=8)
# optimizer = bnb.optim.AdamW(model.parameters(), lr=1e-5, betas=(0.9, 0.999), optim_bits=8)
# 152 1e-5
# cnn 3e-4

div_factor = max_lr/base_lr
num_batches=len(train_loader)
# total_steps=int(num_batches*epochs)+1 # +1 to excluse uptick at the end of onecycle
total_steps=int(np.ceil(num_batches/4)*epochs +1) # /4 for when using grad accumulation
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, total_steps=total_steps, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=100.0, three_phase=True,)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=max_lr, epochs=epochs, steps_per_epoch=num_batches, pct_start=0.45, anneal_strategy='cos', cycle_momentum=True, base_momentum=0.85, max_momentum=0.95, div_factor=div_factor, final_div_factor=10000.0, three_phase=True,)
# gamma = np.exp(np.log(end_lr/start_lr)/(num_batches*epochs)) # for scheduler step every optimizer step
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma) # 0.75(20)-0.9(100)

# pth='/content/res15270kg.pth'
pth='/content/res15270kg05aug.pth' # sps

# pth='/content/drive/MyDrive/frame/vit.pth'
# pth='/content/drive/MyDrive/frame/vit3736.pth'
pth='/content/drive/MyDrive/frame/vit161508aug.pth' # A

# pth='/content/drive/MyDrive/frame/inception1.pth'
pth='/content/drive/MyDrive/frame/inception363505aug.pth' # ty


# # to continue training
# tp, modelsd, optimsd, schedsd = torch.load(pth).values()
# model.load_state_dict(modelsd)
# optimizer.load_state_dict(optimsd)
# scheduler.load_state_dict(schedsd)



In [ ]:

import time
start = time.time()

for t in range(tp,epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    lr=optimizer.param_groups[0]["lr"]
    print(lr)
    train_ls = strain(train_loader, model, loss_fn, optimizer, scheduler)
    correct, test_loss = test(test_loader, model, loss_fn)
    train_lst.extend(train_ls)
    test_lst.append(test_loss)
    acc_lst.append(correct)

    checkpoint = { # https://discuss.pytorch.org/t/saving-model-and-optimiser-and-scheduler/52030
    'epoch': t+1,
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict(),
    'lr_sched': scheduler.state_dict()}
    torch.save(checkpoint, pth)
    # torch.save(model.state_dict(), pth)

# print("Done!")

# end = time.time()
# print("time: ",end - start)

# print(len(train_lst), len(test_lst))
# import matplotlib.pyplot as plt
# plt.plot(train_lst)
# plt.plot(np.linspace(0,len(train_lst),len(test_lst)), test_lst)
# plt.show()
# plt.plot(acc_lst)
# plt.show()
# plt.close()


# torch.save(model.state_dict(), "model.pth")

# resnet 18, 60/61 38.4%, 528s
# resnet 18, 58/61 39.8%, 523s
# resnet 18 compile , 58/61 40.4%, 555s
# resnet 18 compile augment , 58/61 36.4%, 1941 # augment on cpu, takes longer
# resnet 18 augment lr3e-4:3e-3, 58/61 37.7%, 1863s
# resnet 18 augment 10epoch lr1e-5:3e-4, 58/61 33.5%, 3387s
# resnet 18 compile lr1e-5:3e-4, 58/61 35.0%, 493s
# resnet 18 compile scratch lr1e-5:1e-3, 58/61 26.8%, 475s
# resnet 18 compile lr1e-5:1e-3, 55/61 47.3%, 480s
# resnet 18 compile lr1e-5:1e-3, 52/61 51.7% 503s
# resnet 18 compile lr1e-5:1e-3, unfreeze 51.0%, 550s
# resnet 18 compile lr1e-5:1e-4, unfreeze 52.7%, 518s
# resnet 34 compile lr1e-5:1e-4, unfreeze bitsadamW batch16*4
# resnet 152 compileoverhead lr3e-7:3e-6, bitsadamW batch16*4 ckpt 53.8%, 2066s
# resnet 152 from53.8% augment+cutout lr3e-7:3e-6, 53.8%, 2088s
# resnet 152 comile augment+cutout lr1e-5 /4 1epoch 48.3%, 454s
# resnet 152 comile augment+cutout lr1e-5 1epoch 48.4%, 446s
# resnet 152 comilemaxautotue augment+cutout lr1e-5 1epoch 47.7%, 448
# resnet 152 clipclean comilemax augment+cutout lr1e-5 10epoch 45.1%, 1585 *2
# resnet 152 clipclean comilemax augmax lr1e-6:3e-5 20epoch 
# resnet 152 clipclean compilemax lr3e-7:1e-5,  
# resnet 152 70k augment compilemax lr3e-7:3e-6 5epoch, 65.4% 11585s*5/3=19300s = 5h20m


# vit b16 lr1e-5 5epochs 41.3%, 466s # 4.4ram, 5.5vram
# vit l16 lr1e-5 5epochs # 32.0%, 1242s 4.5ram, 8.0vram
# vit l16 lr3e-7;1e-5 5epochs # 45.4%, 1315s 4.5ram, 8.0vram
# vit l32 lr1e-5 5epochs # .ram, .vram
# vit_large_patch16_384
# vit_base_patch16_224 maxcompile nockpt lr3e-7;1e-5 5epochs # 45.2%, 2272s 5.3ram, 11.0vram
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 1e-5, 1e-2 explode
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 1e-6, 1e-5 low acc, test nan
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 3e-7, 3e-6 4/5 epoch50%
# vit_base_patch16_224 maxcompile nockpt gradacc batch16 1e-6, 1e-5 0.5aug



# inception
# inception og 10kclean 1e-7, 3e-5 batch64 nockpt 5epochs 0.020348  51.4%, 0.024312 20m; 1st 40.5% max 52.1
# inception hid 10kclean 1e-7, 3e-5 batch64 nockpt 1/5epochs 0.019635 53.6%, 0.024120 1st 32.8%, max 53.6%
# inception og 10kclean 1e-7, 1e-4 batch64 nockpt 2/10epochs 0.020348  0.020923 39.9%, 0.026105
# inception og 70kg 1e-7 nope
# inception og 70kg 3e-6 3e-5 17%
# inception og 70kg 1e-6, 1e-5
# inception og 70kg 3e-6 3e-5 0.5aug





Epoch 1
-------------------------------
3.000000000000001e-07
loss: 0.087659 [    0/70000]
loss: 0.085325 [   16/70000]
loss: 0.089948 [   32/70000]
loss: 0.094672 [   48/70000]
loss: 0.083321 [   64/70000]
loss: 0.095851 [   80/70000]
loss: 0.095427 [   96/70000]
loss: 0.072946 [  112/70000]
loss: 0.084744 [  128/70000]
loss: 0.088063 [  144/70000]
loss: 0.082541 [  160/70000]
loss: 0.085507 [  176/70000]
loss: 0.089179 [  192/70000]
loss: 0.082221 [  208/70000]
loss: 0.092031 [  224/70000]
loss: 0.092860 [  240/70000]
loss: 0.081482 [  256/70000]
loss: 0.090427 [  272/70000]
loss: 0.087398 [  288/70000]
loss: 0.086880 [  304/70000]
loss: 0.077834 [  320/70000]
loss: 0.082740 [  336/70000]
loss: 0.093418 [  352/70000]
loss: 0.092308 [  368/70000]
loss: 0.087848 [  384/70000]
loss: 0.084858 [  400/70000]
loss: 0.074539 [  416/70000]
loss: 0.109645 [  432/70000]
loss: 0.086421 [  448/70000]
loss: 0.084082 [  464/70000]
loss: 0.090399 [  480/70000]
loss: 0.100385 [  496/70000]
loss: 0.07

In [ ]:
correct, test_loss = test(test_loader, model, loss_fn)
print(correct, test_loss)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Accuracy: 46.1%, Avg loss: 0.099935


In [ ]:
# pytorch release gpu ram after training
# https://discuss.pytorch.org/t/free-all-gpu-memory-used-in-between-runs/168202/2

# torch.cuda.empty_cache()

# correct, test_loss = test(test_loader, model, loss_fn)
# train_lst.extend(train_ls)
# test_lst.append(test_loss)
# acc_lst.append(correct)



In [ ]:
# @title save
from google.colab import drive
drive.mount('/content/drive')

# # pth='/content/drive/MyDrive/frame/vit.pth'
# pth='/content/drive/MyDrive/frame/res55.pth'
# pth='/content/drive/MyDrive/frame/resnet152.pth'
# pth='/content/drive/MyDrive/frame/inception.pth'
# pth='/content/inception2.pth'

pth='/content/res15270kg.pth'


tp, modelsd, optimsd, schedsd = torch.load(pth).values()
model.load_state_dict(modelsd)
optimizer.load_state_dict(optimsd)
scheduler.load_state_dict(schedsd)


# checkpoint = { # https://discuss.pytorch.org/t/saving-model-and-optimiser-and-scheduler/52030
# 'epoch': t,
# 'model': model.state_dict(),
# 'optimizer': optimizer.state_dict(),
# 'lr_sched': scheduler.state_dict()}
# torch.save(checkpoint, pth)




# torch.save(model.state_dict(), pth)
# model.load_state_dict(torch.load(pth))
# # model.load_state_dict(torch.load("model.pth"))

# !gdown 1---4fdFbOUBTrS-VP5Va6pKowfgoU2UN -O inception2.pth
# !gdown 1visTNvWmnuV7jAm2TBiAIIrNjbOAi1Fv -O resnet152.pth

# # t, modelsd, optimsd, scheduler = torch.load('/content/drive/MyDrive/frame/resnet152.pth').values()
# t, modelsd, optimsd, scheduler = torch.load('/content/resnet152.pth').values()
# model.load_state_dict(modelsd)
# # optimizer.load_state_dict(optimsd)

# # matt152 # https://drive.google.com/file/d/1MQ0xLfHbio458uEVbn2VyMpD3bij2A4J/view?usp=sharing
# !gdown 1MQ0xLfHbio458uEVbn2VyMpD3bij2A4J -O res152.pth
# model.load_state_dict(torch.load("res152.pth"))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('8013'), PosixPath('http'), PosixPath('//172.28.0.1')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

NameError: ignored

In [ ]:
# @title trash
# dir='/content/google_street_view'

# # data = datasets.ImageFolder(dir, transform=transform)
# data = datasets.ImageFolder(dir, transform=None)
# torch.manual_seed(0)
# train_data, test_data = torch.utils.data.random_split(data, [.85,.15])

# # train_data = DatasetWrap(train_data, TrainTransform()) # apply data augmentation to train dataset only
# train_data = DatasetWrap(train_data, transform)
# test_data = DatasetWrap(test_data, transform)


# batch_size = 16 # 64
# num_batches=int(np.ceil(len(train_data)/batch_size))

# train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
# # data, train_data, test_data = None, None, None


# dataiter = iter(train_loader)
# images, labels = next(dataiter)
# imshow(torchvision.utils.make_grid(images))
# dataiter=None


%matplotlib inline 
from matplotlib import pyplot as plt

# dataiter = iter(train_data)


# img,label = next(iter(sample_ds))
i=5
# print(len(test_data))
# img,label=test_data[i]
img,label=sample_ds[i]
print(img.shape)
# print(type(img))
model.eval()
pred=model(img.unsqueeze(0).to(device))
pred_probab = nn.Softmax(dim=1)(pred)
y_pred = pred_probab.argmax(1)
print(pred_probab)
print("pred: ",y_pred.item())
# print(img)
# image=images[0]

print("actual: ",label)
plt.imshow(np.transpose(img, (1, 2, 0)))
plt.show()


In [ ]:
# @title trash

model.eval()
count=0
i=138
rong_lst=[]

while count<20:
    img,label=test_data[i]
    pred=model(img.unsqueeze(0).to(device))
    pred_probab = nn.Softmax(dim=1)(pred)
    y_pred = pred_probab.argmax(1)
    if y_pred.item() != label:
        print("pred: ",y_pred.item(),", actual: ",label)
        # plt.imshow(np.transpose(img, (1, 2, 0)))
        # plt.show()
        imshow(img)
        rong_lst.append(img)
        count+=1
    i+=1


# 20/137 wrong


In [ ]:
print(i)
# imshow(torchvision.utils.make_grid(images,nrow=4))
# print(torch.stack(rong_lst).shape)
# print(len(rong_lst))
# print(rong_lst[0].shape)
imshow(torchvision.utils.make_grid(torch.stack(rong_lst),nrow=4))
